In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

# 1. Get course codes from program page

In [2]:
url = "https://laurentian.ca/program/mechanical-engineering/details"

In [3]:
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [4]:
start = page_soup.find("div", {"id": "options"}).find("strong", text="Mechanical Engineering Technical Electives – List A")
start

<strong>Mechanical Engineering Technical Electives – List A</strong>

In [6]:
import bs4

course_titles = []
counter = 0

while start != "Complementary Studies Electives":
    if type(start) is bs4.element.NavigableString:
        course_title = " ".join(re.split("\s+", start.strip()))
        if bool(re.match("[A-Z]{4} [0-9]{4}E", course_title)):
            course_titles.append(course_title)
            print(course_titles[-1])
            counter += 1
    start = start.next_element

COSC 2307E Database Programming
COSC 2406E Assembly Language Programming
ENGR 3126E Occupational Health Engineering
ENGR 3157E Operations Research
ENGR 3307E Materials Handling
ENGR 3437E Water and Wastewater Treatment
ENGR 3466E Environmental Impact of Process Industries
ENGR 3477E Adsorption and Membrane Processes in Pollution Control
ENGR 4157E Power Machines
ENGR 4166E Aerospace Engineering Design
ENGR 4187E Solid Waste Processing and Recycling in the Minerals Industry
ENGR 4366E Automation and Reliability of Mining Equipment
MATH 2037E Advanced Calculus
MATH 2057E Linear Algebra II
PHYS 2006E Introduction to Electricity and Electronics
PHYS 3046E Mechanical Properties of Solids and Fluids
BIOL 1506E Biology I
BIOL 2356E Principles of Ecology
CHMI 1007E General Chemistry II
ENSC 1406E Earth’s Environmental Systems
GEOL 1006E Introductory Geology I
PHYS 2606E Biophysics of Fluids
PHYS 2616E Physics of Hearing and Vision


In [7]:
counter

23

# 2. Scrape course descriptions from courses website

In [8]:
url = "https://laurentian.ca/courses"

In [18]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [19]:
driver.get(url)

In [20]:
from selenium.common.exceptions import NoSuchElementException

course_names = []
course_descs = []
course_codes = []
counter = 0

for course_title in course_titles:
    
    #type in search box
    driver.find_elements_by_tag_name("input")[1].send_keys(course_title)
    time.sleep(1)
    
    #click on course link
    try:
        driver.find_element_by_link_text("More Info").click()
        time.sleep(3)
    except NoSuchElementException:
        driver.find_elements_by_tag_name("input")[1].clear()
        time.sleep(1)
        driver.find_elements_by_tag_name("input")[1].send_keys(" ".join(course_title.split()[:2]))
        time.sleep(1)
        
        try:
            driver.find_element_by_link_text("More Info").click()
            time.sleep(3)
        except NoSuchElementException:
            print("Cannot find ", course_title)
            driver.find_elements_by_tag_name("input")[1].clear()
            time.sleep(1)
            continue
    
    #switch to new tab
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    
    #get html and scrape
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "region region-content"}).find("div", {"class": "container"})
    
    course_name = container.h1.text.strip()
    course_desc = container.find("p").text.strip().split("PREREQ:")[0]
    course_code = " ".join(course_title.split()[:2])
    
    course_names.append(course_name)
    course_descs.append(course_desc)
    course_codes.append(course_code)
    
    counter += 1
    print("scraped ", course_codes[-1])
    
    #close tab
    driver.close()
    time.sleep(1)
    
    #switch to original tab
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)
    
    #clear search box
    driver.find_elements_by_tag_name("input")[1].clear()
    time.sleep(1)
    
print(counter)

scraped  COSC 2307E
scraped  COSC 2406E
scraped  ENGR 3126E
scraped  ENGR 3157E
scraped  ENGR 3307E
scraped  ENGR 3437E
scraped  ENGR 3466E
scraped  ENGR 3477E
Cannot find  ENGR 4157E Power Machines
scraped  ENGR 4166E
scraped  ENGR 4187E
scraped  ENGR 4366E
scraped  MATH 2037E
scraped  MATH 2057E
scraped  PHYS 2006E
scraped  PHYS 3046E
scraped  BIOL 1506E
scraped  BIOL 2356E
scraped  CHMI 1007E
scraped  ENSC 1406E
scraped  GEOL 1006E
scraped  PHYS 2606E
scraped  PHYS 2616E
22


# 3. Inspect and write to CSV

In [21]:
course_codes

['COSC 2307E',
 'COSC 2406E',
 'ENGR 3126E',
 'ENGR 3157E',
 'ENGR 3307E',
 'ENGR 3437E',
 'ENGR 3466E',
 'ENGR 3477E',
 'ENGR 4166E',
 'ENGR 4187E',
 'ENGR 4366E',
 'MATH 2037E',
 'MATH 2057E',
 'PHYS 2006E',
 'PHYS 3046E',
 'BIOL 1506E',
 'BIOL 2356E',
 'CHMI 1007E',
 'ENSC 1406E',
 'GEOL 1006E',
 'PHYS 2606E',
 'PHYS 2616E']

In [22]:
course_names

['Database Programming',
 'Assembly Language Programming',
 'Occupational Health Engineering',
 'Operations Research',
 'Materials Handling',
 'Water and Wastewater Treatment',
 'Environmental Impact of Process Industries',
 'Adsorption and Membrane Processes in Polution Control',
 'Aerospace Engineering Design',
 'Solid Waste Processing and Recycling in the Minerals Industry',
 'Automation and Reliability of Mining Equipment',
 'Advanced Calculus',
 'Linear Algebra II',
 'Introduction to Electricity and Electronics',
 'Mechanical Properties of Solids and Fluids',
 'Biology I',
 'Principles of Ecology',
 'General Chemistry II',
 "Earth's Environmental Systems",
 'Introductory Geology I',
 'Biophysics of Fluids',
 'Physics of Hearing and Vision']

In [24]:
course_descs

['This course consists of data modelling, the E-R data model, normalization, database schema, SQL, single and multiple table queries, subqueries, updates, embedded SQL and report generation. ',
 'This is an introduction to basic computer organization and instruction set architecture. Topics considered include: the instruction execution cycle, an overview of the assembly process, data representation, addressing modes, arrays and strings, translation of high-level language control structures, procedures, parameter passing, and recursion, macros and conditional assembly, interrupts and input/output, interfacing to high level languages, floating point processor architecture and instructions and code optimization techniques. ',
 'This course examines occupational health problems encountered in industry; particularly those due to dusts, gases, ionizing and non-ionizing radiation, noise, heat stress and vibration. Methods of establishing safe threshold limit values for these human stressors a

In [25]:
len(list(dict.fromkeys(course_codes)))

22

In [26]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,COSC 2307E,Database Programming,"This course consists of data modelling, the E-..."
1,COSC 2406E,Assembly Language Programming,This is an introduction to basic computer orga...
2,ENGR 3126E,Occupational Health Engineering,This course examines occupational health probl...
3,ENGR 3157E,Operations Research,This course deals with the fundamentals of app...
4,ENGR 3307E,Materials Handling,This course examines materials handling techno...
5,ENGR 3437E,Water and Wastewater Treatment,This course examines unit operations and unit ...
6,ENGR 3466E,Environmental Impact of Process Industries,This course examines general concepts of indus...
7,ENGR 3477E,Adsorption and Membrane Processes in Polution ...,Topics include adsorption as a treatment proce...
8,ENGR 4166E,Aerospace Engineering Design,This course examines introductory concepts rel...
9,ENGR 4187E,Solid Waste Processing and Recycling in the Mi...,This course introduces students to solid waste...


In [27]:
df.to_csv('Laurentian_MechEng_Technical_Electives_Courses.csv', index = False)

In [28]:
driver.quit()